In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

# xpath 추출 함수
def get_xpath():
    import requests
    from lxml import html

    for i in range(0, 1200):
        page = requests.get(item_url[i])
        root = html.fromstring(page.text)
        tree = root.getroottree()
    
        # 상품 제목 xpath
        title='//h3[contains(text(), "{}")]'.format(item_title[i][len(item_title[i])-5:])
        title_result = root.xpath(title)
    
        # 상품 이미지 xpath
        img_result = root.xpath('//img[@alt="상품 이미지1"]')
    
        # 상품 가격 xpath
        price='//*[. = "{}"]'.format(item_price[i])
        price_result = root.xpath(price)
    
        try:
            title_xpath.append(tree.getpath(title_result[0]))
        
        except Exception as ex:
            title_xpath.append('N/A')
            
        try:
            img_xpath.append(tree.getpath(img_result[0]))
        except Exception as ex:
            img_xpath.append('N/A')
            
        try:
            price_xpath.append(tree.getpath(price_result[0]))
        except Exception as ex:
            price_xpath.append('N/A')
    

# 상품 정보 리스트
item_url=[]
item_title=[]
item_price=[]
item_img=[]
item_rank=[]
item_category=[]
title_xpath=[]
img_xpath=[]
price_xpath=[]

# 12개 카테고리별 best 100 수집
for i in range(1,13):
    
    # url 할당 조건
    if i < 10:
        url = 'http://mobile.gmarket.co.kr/Best/Category?code=G0'+str(i)
        response = requests.get(url)
    else:
        url = 'http://mobile.gmarket.co.kr/Best/Category?code=G'+str(i)
        response = requests.get(url)
    
    # 정상 접속
    if response.status_code == 200:

        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        select_data = soup.select('li[id^=selected] > a')
        rank=1
    
        # 데이터 수집
        for j in select_data:
            # 상품 클릭 url
            item_url.append(j.get('href'))
            # 상품 제목
            item_title.append(j.find("strong").text)
            # 상품 가격
            item_price.append(j.find("em").text)
            # 상품 이미지
            item_img.append(j.find("img").get("data-original"))
            # 상품 순위
            item_rank.append(rank)
            rank+=1
            # 상품 카테고리
            if i == 1:
                item_category.append("패션의류")
            elif i == 2:
                item_category.append("신발잡화")
            elif i == 3:
                item_category.append("화장품헤어")
            elif i == 4:
                item_category.append("유아동출산")
            elif i == 5:
                item_category.append("스포츠자동차")
            elif i == 6:
                item_category.append("컴퓨터전자")
            elif i == 7:
                item_category.append("식품")
            elif i == 8:
                item_category.append("생활주방건강")
            elif i == 9:
                item_category.append("가구침구")
            elif i == 10:
                item_category.append("도서음반")
            elif i == 11:
                item_category.append("여행")
            elif i == 12:
                item_category.append("e쿠폰")
                
        
    # 접속 에러    
    else : 
         print(response.status_code)

# xapth get 함수 호출
get_xpath()
        
# 데이터 프레임 생성 후 csv 파일 생성        
df=pd.DataFrame(zip(item_rank, item_title, item_price, item_url, item_img, item_category, title_xpath, img_xpath, price_xpath),
                columns=['rank', 'title', 'price', 'url', 'img', 'category', 'title_xpath', 'img_xpath', 'price_xpath'])
df.to_csv('/root/data/gmaket.csv', encoding='utf-8', index=None)